In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlretrieve, quote
import os

In [2]:
# URL inicial onde a Receita Federal divulga a Arrecadação por Estado
url_arrecadacao = 'https://receita.economia.gov.br/dados/receitadata/arrecadacao/arrecadacao-por-estado'

# Pasta onde serão guardadas as planilhas
pasta='../planilhas/'
try:
    os.mkdir(pasta)
    print(f'"Pasta {pasta} criada') 
except FileExistsError:
    print(f'"Pasta {pasta} já existe') 

"Pasta ../planilhas/ já existe


In [3]:
# Para cada ano a Receita Federal adotou um padrão diferente para o nome do arquivo
# Essa função padronizará o nome para o formato 'AAAA-MM-arrecadacao-uf'
def padroniza_nome_arquivo(nome_arquivo, ano):
    
    nome_arquivo = nome_arquivo.lower()
    extensao = nome_arquivo.split('.',1)[-1]
    
    #identifica o mês
    if('jan' in nome_arquivo):
        mes='01'
    elif('fev' in nome_arquivo):
        mes='02'
    elif('mar' in nome_arquivo):
        mes='03'
    elif('abr' in nome_arquivo):
        mes='04'
    elif('mai' in nome_arquivo):
        mes='05'
    elif('jun' in nome_arquivo):
        mes='06'
    elif('jul' in nome_arquivo):
        mes='07'
    elif('ago' in nome_arquivo):
        mes='08'
    elif('set' in nome_arquivo):
        mes='09'
    elif('out' in nome_arquivo):
        mes='10'
    elif('nov' in nome_arquivo):
        mes='11'
    elif('dez' in nome_arquivo):
        mes='12'
    else:
        mes='MES-NAO-IDENTIFICADO'
        print(f'{mes} no arquivo {nome_arquivo}')
        
    nome_arquivo_padrao = f'{ano}-{mes}-arrecadacao-uf.{extensao}'
    
    return nome_arquivo_padrao

In [4]:
# Função utilizada para abrir as diversas URL necessárias e devolver um soup parser
def bs_parser(url):
    u = urlopen(url)
    html = u.read().decode('utf-8')
    u.close
    soup = BeautifulSoup(html, "html.parser")
    return soup 

In [7]:
# Carrega a URL Arrecação da Receita Federal
soup=bs_parser(url_arrecadacao)

# Anos escopo do trabalho
ano_inicial = '2004'
ano_final   = '2019'

print(f'::: INICIANDO DOWNLOAD :::')
padrao_link = 'a[href^="https://receita.economia.gov.br/dados/receitadata/arrecadacao/arrecadacao-por-estado/arrecadacao-uf-"]'

contador_global = 0
# Seleciona todos os elementos que contenham links para os anos em que a arecadação está disponível
for link in soup.select(padrao_link):
    url_ano = link.get('href')
    ano = url_ano.rsplit('-',1)[-1]
    # Apenas o período de interesse
    if (ano < ano_inicial or ano > ano_final):
        continue
        
    print(f'::: ANO {ano} :::')
    print(url_ano)
    soup_ano = bs_parser(url_ano)
    contador=0
        
    # Identifica todos os arquivos disponíveis com arrecadação para determinado ano
    for link in soup_ano.select(padrao_link):
        url_mes = link.get('href')
        
        # Verifica se a extensão é do link é '.ods' ou '.xls', acrescidos do sufixo 'ods/view'
        if not any(url_mes.endswith(x) for x in ['ods/view','.xls/view']):
            continue
        
        url_minusculas = url_mes.lower()
        # Ignora os arquivos consolidados de todos os meses do ano
        if ('jan-dez' in url_minusculas) or ('janeiro-dezembro' in url_minusculas) or ('jan-a-dez' in url_minusculas):
            continue
            
        soup_mes = bs_parser(url_mes)
        for link in soup_mes.select('a[href^=""]'):
            url_arquivo = link.get('href')

            # Verifica se a extensão é do link é '.ods' ou '.xls'
            if not any(url_arquivo.endswith(x) for x in ['.ods','.xls']):
                continue
           
            arquivo = url_arquivo.rsplit('/', 1)[-1]
            
            #Concatena a pasta com nome do arquivo (planilha) padronizado
            nome_arquivo = pasta + padroniza_nome_arquivo(arquivo, ano)
            #print(f"Baixando {nome_arquivo} de {url_arquivo}")
        
            # Baixa o arquivo e grava na pasta com nome padronizado
            urlretrieve(url_arquivo, nome_arquivo)
            contador += 1
            contador_global += 1
            
    print(f'::: FIM {ano}: {contador} arquivo(s) recuperado(s) :::')
print(f'::: Total de {contador_global} arquivo(s) recuperado(s) :::')
print(f'::: FIM :::')

::: INICIANDO DOWNLOAD :::
::: ANO 2019 :::
https://receita.economia.gov.br/dados/receitadata/arrecadacao/arrecadacao-por-estado/arrecadacao-uf-2019
::: FIM 2019: 12 arquivo(s) recuperado(s) :::
::: ANO 2018 :::
https://receita.economia.gov.br/dados/receitadata/arrecadacao/arrecadacao-por-estado/arrecadacao-uf-2018
::: FIM 2018: 12 arquivo(s) recuperado(s) :::
::: ANO 2017 :::
https://receita.economia.gov.br/dados/receitadata/arrecadacao/arrecadacao-por-estado/arrecadacao-uf-2017
::: FIM 2017: 12 arquivo(s) recuperado(s) :::
::: ANO 2016 :::
https://receita.economia.gov.br/dados/receitadata/arrecadacao/arrecadacao-por-estado/arrecadacao-uf-2016
::: FIM 2016: 12 arquivo(s) recuperado(s) :::
::: ANO 2015 :::
https://receita.economia.gov.br/dados/receitadata/arrecadacao/arrecadacao-por-estado/arrecadacao-uf-2015
::: FIM 2015: 12 arquivo(s) recuperado(s) :::
::: ANO 2014 :::
https://receita.economia.gov.br/dados/receitadata/arrecadacao/arrecadacao-por-estado/arrecadacao-uf-2014
::: FIM 201